# 5D uncertain Dubins car model

In [1]:
# If first time running this code on a personal computer
# using Pkg
# Pkg.instantiate()

### Script / SCP Initialization

In [4]:
using LinearAlgebra
using Ipopt
using JuMP
using NLsolve
using Plots

include("./Models/dubins5D.jl")
include("./SCP/scp_problem.jl")
include("./utils/dubins_plotting.jl")

# Number of time-discretization points and maximum number of GuSTO iterations
N, max_it = 41, 15

# Defining the model, the initial strategy and each convex subproblem
model         = Dubins5D()
μ_p, Σ_p, U_p = initialize_trajectory(model,N)
scp_problem   = SCPProblem(model, N, μ_p, Σ_p, U_p)

# Defining SCP parameters
(Delta0, omega0, omegamax, epsilon,
        convergence_threshold) = get_initial_scp_parameters(model)

(50.0, 500.0, 1.0e6, 0.001, 0.001)

## SCP algorithm

In [23]:
# Defining penalization weights, trust-region radius and the list of solutions
@time begin

include("./Models/dubins5D.jl")
include("SCP/scp_problem.jl")
μ_p,   Σ_p,   U_p   = initialize_trajectory(model, N)
μ,     Σ,     U     = copy(μ_p), copy(Σ_p), copy(U_p)
μ_all, Σ_all, U_all = [], [], []
push!(μ_all, copy(μ))
push!(Σ_all, copy(Σ))
push!(U_all, copy(U))

Delta = Delta0

# GuSTO loop
success, it = false, 1
while (it < max_it) && 
      !(success && 
         (convergence_metric(model, μ,U, μ_p,U_p) +
          convergence_metric(model, μ_all[end-2],U_all[end-2], μ_p,U_p)) 
                < convergence_threshold)
    println("-----------\nIteration $it\n-----------")
    
    # Store the solution at the previous step and the linearized dynamics
    μ_p, Σ_p, U_p                                        = copy(μ), copy(Σ), copy(U)
    model.b, model.b_dx, model.b_du, model.σ, model.σ_dx = compute_dynamics(model, μ_p, U_p)
    
    # Define the convex subproblem
    reset_problem(        scp_problem, model)
    set_parameters(       scp_problem, model, μ_p, U_p, omega0, Delta)
    define_nonconvex_cost(scp_problem, model)
    define_constraints(   scp_problem, model)
    
    # Solve the convex subproblem
    JuMP.optimize!(scp_problem.solver_model)
    println("optimized: ", termination_status(scp_problem.solver_model))
    
    μ_sol, Σ_sol, U_sol = JuMP.value.(scp_problem.μ), JuMP.value.(scp_problem.Σ), JuMP.value.(scp_problem.U)
    
    # -----------
    # SCP
    println("Accept solution.")
    μ, Σ, U = copy(μ_sol), copy(Σ_sol), copy(U_sol)
    
    Delta = 0.99 * Delta

    if (it > 2) # needs at least 3 iterations to check convergence
        success = true
    else
        success = false
    end

    # Collecting the solution at each iteration
    push!(μ_all,copy(μ))
    push!(Σ_all,copy(Σ))
    push!(U_all,copy(U))
    it += 1
    
    println("(1-step) metric = $(convergence_metric(model,μ,U,μ_p,U_p))")
end

println(">>> Finished <<<")

μ_f,Σ_f,U_f,μ_fp,Σ_fp,U_fp = μ_all[end],Σ_all[end],U_all[end],μ_all[end-1],Σ_all[end-1],U_all[end-1]
if satisfies_trust_region_constraints(scp_problem, model, μ_f,Σ_f,U_f,μ_fp,Σ_fp,U_fp, Delta)
    println(">>>>> Satisfies trust region constraint.")
end

end 

-----------
Iteration 1
-----------


optimized: LOCALLY_INFEASIBLE
Accept solution.
(1-step) metric = 2.040548109854185
-----------
Iteration 2
-----------


optimized: LOCALLY_SOLVED
Accept solution.
(1-step) metric = 5.7421841325029845
-----------
Iteration 3
-----------


optimized: LOCALLY_SOLVED
Accept solution.
(1-step) metric = 3.409217676045821
-----------
Iteration 4
-----------


optimized: LOCALLY_SOLVED
Accept solution.
(1-step) metric = 1.8003372055601425
-----------
Iteration 5
-----------


...


optimized: LOCALLY_SOLVED
Accept solution.
(1-step) metric = 0.03239086830076095
-----------
Iteration 8
-----------


optimized: LOCALLY_SOLVED
Accept solution.
(1-step) metric = 0.005021970741826527
-----------
Iteration 9
-----------


optimized: LOCALLY_SOLVED
Accept solution.
(1-step) metric = 0.0007857782964299765
-----------
Iteration 10
-----------


optimized: LOCALLY_SOLVED
Accept solution.
(1-step) metric = 0.00012167306573109179
>>> Finished <<<


>>>>> Satisfies trust region constraint.
  4.658326 seconds (8.28 M allocations: 451.709 MiB, 2.04% gc time, 20.43% compilation time)


# Plots

In [19]:
include("./utils/dubins_plotting.jl")
idx = [1,2]
fig = plt_solutions(scp_problem, model, μ_all, Σ_all, U_all,
                B_plot_ellipses_final_traj=false, B_manually_set_lims=true, idx=idx)

include("./Models/dubins5D.jl")
X_MC = simulate_monte_carlo(model, U_f, N_MC=10000)
plot_MC(fig, X_MC[:,:,1:1000], idx=idx)

ax = plt.gca()
ax.plot(μ_all[end][idx[1],:], μ_all[end][idx[2],:],
                    label="Final", linewidth=1, color="b", linestyle="dashed", marker="o")

#plt.legend(loc='lower right', fontsize=20, labelspacing=0.)
plt.draw()
plt.xlabel(L"x", fontsize=20)
plt.ylabel(L"y", rotation=0, labelpad=20, fontsize=20)
plt.savefig("figs/freeflyer/trajs.png",bbox_inches="tight",dpi=350)

print("Done Plotting")

Done Plotting

sys:1: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [9]:
include("./utils/dubins_plotting.jl")
plt_all_lin_velocities(μ_all, false)

ax = plt.gca()
ax.plot(1:(size(μ_all[end])[2]), μ_all[end][4,:],
                    label="Final", linewidth=1, color="b", linestyle="dashed", marker="o")
# plot_MC_1dim(fig, X_MC, idx=5)

plt.title(L"Linear Velocities $v$", pad=10)

# plt.savefig("figs/freeflyer/vels_lin.png",bbox_inches="tight",dpi=350)

print("Done Plotting")

Done Plotting

In [12]:
include("./utils/dubins_plotting.jl")
plt_all_ang_velocities(μ_all, false)

ax = plt.gca()
ax.plot(1:(size(μ_all[end])[2]), μ_all[end][5,:],
                    label="Final", linewidth=1, color="b", linestyle="dashed", marker="o")
# plot_MC_1dim(fig, X_MC, idx=5)

plt.title(L"Angular Velocities $\omega$", pad=10)

plt.savefig("figs/freeflyer/vels_ang.png",bbox_inches="tight",dpi=350)

print("Done Plotting")

In [13]:
include("./utils/dubins_plotting.jl")
plt_all_lin_controls(U_all, false)

ax = plt.gca()
ax.plot(1:(size(U_all[end])[2]), U_all[end][1,:],
                    label="Final", linewidth=1, color="b", linestyle="dashed", marker="o")
# plot_MC_1dim(fig, X_MC, idx=5)

plt.title(L"Linear Controls $a_v$", pad=10)

plt.savefig("figs/freeflyer/controls_lin.png",bbox_inches="tight",dpi=350)

print("Done Plotting")

Done Plotting

In [14]:
include("./utils/dubins_plotting.jl")
plt_all_ang_controls(U_all, false)

ax = plt.gca()
ax.plot(1:(size(U_all[end])[2]), U_all[end][2,:],
                    label="Final", linewidth=1, color="b", linestyle="dashed", marker="o")
# plot_MC_1dim(fig, X_MC, idx=5)

plt.title(L"Angular Controls $a_{\omega}$", pad=10)

plt.savefig("figs/freeflyer/controls_ang.png",bbox_inches="tight",dpi=350)

print("Done Plotting")

Done Plotting

In [15]:
size(X_MC)

(5, 41, 10000)

In [16]:
length(model.obstacles)

4

In [17]:
# Evaulate nb of collided trajectories
x_dim, N, N_MC = size(X_MC)
nb_in_obs      = zeros(N_MC)
for k = 1:N
    for i = 1:N_MC
        for obs_i = 1:length(model.obstacles)
            x_ki = X_MC[:,k,i]
            po, ro = model.obstacles[obs_i]

            if (x_ki[1]-po[1])^2+(x_ki[2]-po[2])^2 < ro^2
                nb_in_obs[i] = 1
            end
        end
    end
end
print("nb in obs = $(sum(nb_in_obs))/$N_MC ==> percentage = $(100.0*sum(nb_in_obs) / N_MC)%")

nb in obs = 614.0/10000 ==> percentage = 6.14%